In [20]:
import gzip
import opener
import pandas as pd
import io
import requests
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from collections import Counter

In [3]:
url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2018/full.csv.gz"
req = requests.get(url)

In [4]:
t = req.content

In [5]:
s = gzip.decompress(t)

In [6]:
df = pd.read_csv(io.BytesIO(s))

C:\Users\dargo\AppData\Local\Temp\ipykernel_25512\798997524.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.BytesIO(s))


In [7]:
df

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Appartement,73.0,4.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
1,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
2,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,S,sols,NaN,NaN,949.0,5.408041,46.255562
3,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
4,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,51.0,2.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335287,2018-1434017,2018-12-28,1,Vente,1192307.0,1.0,NaN,PAS CHOISEUL,2012,75002.0,...,Appartement,150.0,4.0,NaN,NaN,NaN,NaN,NaN,2.335139,48.867253
3335288,2018-1434018,2018-12-03,1,Vente,383000.0,12.0,NaN,RUE POISSONNIERE,7561,75002.0,...,Appartement,34.0,1.0,NaN,NaN,NaN,NaN,NaN,2.347989,48.869018
3335289,2018-1434019,2018-12-28,1,Vente,746000.0,18.0,NaN,RUE STE CROIX BRETONNERIE,8548,75004.0,...,Local industriel. commercial ou assimilé,60.0,0.0,NaN,NaN,NaN,NaN,NaN,2.356566,48.858267
3335290,2018-1434020,2018-12-13,1,Adjudication,645000.0,9.0,NaN,BD MORLAND,6559,75004.0,...,Appartement,54.0,2.0,NaN,NaN,NaN,NaN,NaN,2.364563,48.848333


In [8]:
df.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [9]:
# suppression valeur fonciere nulle

df.dropna(subset=['valeur_fonciere'], inplace=True)

In [10]:
# Selection des colonnes utiles

dataset = df[['id_mutation',
              'date_mutation',
              'nature_mutation',
              'adresse_nom_voie',
              'code_postal',
              'code_commune',
              'code_departement',
              'nombre_lots',
              'code_type_local',
              'surface_reelle_bati',
              'nombre_pieces_principales',
              'code_nature_culture',
              'code_nature_culture_speciale',
              'surface_terrain',
              'valeur_fonciere'
              ]]

In [11]:
# del df

# Cleaning

In [12]:
# Passage de la colonnes date mutation en datetime puis en seconde pour avoir des int

dataset.loc[:,'date_mutation'] = pd.to_datetime(dataset['date_mutation'], format='mixed')
dataset.loc[:,'date_mutation'] = dataset['date_mutation'].apply(lambda x: x.timestamp())

In [13]:
# Remplissage des valeurs NaN nature_mutation et labelisation

dataset.loc[:,'nature_mutation'] = dataset.nature_mutation.fillna("")

NM = preprocessing.LabelEncoder()
NM.fit(dataset['nature_mutation'])
dataset.loc[:,'nature_mutation'] =  NM.transform(dataset['nature_mutation'])

In [14]:
dataset.loc[:,'adresse_nom_voie'] = dataset.adresse_nom_voie.fillna("")

codex_voie = list()
for i in dataset.adresse_nom_voie.values:
    j = i.split(' ')
    codex_voie.append(j[0])

dataset.loc[:,'prefixe_voie'] = codex_voie

C:\Users\dargo\AppData\Local\Temp\ipykernel_25512\3593151251.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,'prefixe_voie'] = codex_voie


In [15]:
dataset.loc[:,'code_postal'] = dataset.code_postal.fillna(0)

In [16]:
dataset.loc[:,'surface_reelle_bati'] = dataset.surface_reelle_bati.fillna(0)
dataset.loc[:,'surface_terrain'] = dataset.surface_terrain.fillna(0)
dataset.loc[:,'code_type_local'] = dataset.code_type_local.fillna(0)
dataset.loc[:,'nombre_pieces_principales'] = dataset.nombre_pieces_principales.fillna(0)

In [17]:
dataset.loc[:,'code_nature_culture'] = dataset.code_nature_culture.fillna("")
dataset.loc[:,'code_nature_culture_speciale'] = dataset.code_nature_culture_speciale.fillna("")


In [18]:
dataset.isna().sum()

id_mutation                     0
date_mutation                   0
nature_mutation                 0
adresse_nom_voie                0
code_postal                     0
code_commune                    0
code_departement                0
nombre_lots                     0
code_type_local                 0
surface_reelle_bati             0
nombre_pieces_principales       0
code_nature_culture             0
code_nature_culture_speciale    0
surface_terrain                 0
valeur_fonciere                 0
prefixe_voie                    0
dtype: int64

In [19]:
# Sortie de la colonne resultat

## Y = df.valeur_fonciere